# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [ ]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 30.1MB/s]


### Import the required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [ ]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [ ]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Columns to encode
columns_to_encode = ['Brand','Model','FastCharge_KmH', 'RapidCharge','PowerTrain', 'PlugType', 'BodyStyle', 'Segment']
# Apply LabelEncoder
label_encoders = {}
for col in columns_to_encode:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store the encoder for inverse transformations if needed

# Check the result
print(data.head())

   Brand  Model  AccelSec  TopSpeed_KmH  Range_Km  Efficiency_WhKm  \
0     30     46       4.6           233       450              161   
1     31     33      10.0           160       270              167   
2     23      0       4.7           210       400              181   
3      2    101       6.8           180       360              206   
4      9     78       9.5           145       170              168   

   FastCharge_KmH  RapidCharge  PowerTrain  PlugType  BodyStyle  Segment  \
0              50            1           0         2          7        3   
1               7            1           2         2          1        2   
2              36            1           0         2          2        3   
3              32            1           2         2          6        3   
4               2            1           2         2          1        1   

   Seats  PriceEuro  
0      5      55480  
1      5      30000  
2      5      56440  
3      5      68040  
4      4    

## Split the dataset for training and testing in ratio 80:20

In [ ]:
from sklearn.model_selection import train_test_split

# Independent variables (features) and dependent variable (target)
X = data.drop(columns='PriceEuro')  # Replace 'PriceEuro' with your target column
Y = data['PriceEuro']               # Replace 'PriceEuro' with your target column

# Splitting the dataset (80% training, 20% testing)
x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=0.2, random_state=0)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    int64  
 1   Model            103 non-null    int64  
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    int64  
 7   RapidCharge      103 non-null    int64  
 8   PowerTrain       103 non-null    int64  
 9   PlugType         103 non-null    int64  
 10  BodyStyle        103 non-null    int64  
 11  Segment          103 non-null    int64  
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 11.4 KB


## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor


decision_tree_regressor = DecisionTreeRegressor(max_depth=5, min_samples_split=20, max_features=6, random_state=0)
decision_tree_regressor.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, max_features=6, min_samples_split=20,
                      random_state=0)

## Predict the outcomes for X test

In [ ]:
tree_regressor_predict = decision_tree_regressor.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [ ]:

from sklearn.metrics import mean_squared_error, r2_score


decision_tree_r2 = r2_score(y_test, tree_regressor_predict)


mean_squared_error(y_test, tree_regressor_predict)

872926443.8259329

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [ ]:
xgb_regressor = XGBRegressor(max_depth=10, n_estimators=50, learning_rate=0.1, objective='reg:squarederror')

## Predict the outcomes for X test

In [ ]:
xgb_regressor.fit(x_train, y_train)
xgb_regressor_predict = xgb_regressor.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [ ]:
r2_xgboost = r2_score(y_test, xgb_regressor_predict)
r2_xgboost
mean_squared_error(y_test, xgb_regressor_predict)

1219719137.4020991

## Compare the performances of both model for at least three regression metircs

In [ ]:
if decision_tree_r2 <= r2_xgboost:
    print(" XGBoost better")
else:
    print("Decision Tree better")

Decision Tree better
